# Signal trading

1. Illustrative example of signal trading through turtle trading signal and moving average signal

2. Buy-and-hold strategy

NEED TO DO: CORRECT SELL ORDER IN MOVING AVERAGE - AFFECT TRANSACTION COSTS - METRIQUES DRAWDOWN ON BUYnHOLD

## 1. load the libraries and constants

In [1]:
import numpy as np
import pandas as pd
import os

SOURCE_FOLDER = 'Data-processed'
TARGET_FOLDER = 'Data-factor'

DESCRIPTION_FILE = "data_list.csv"
DATA_FILE = "data_content.csv"

INVESTMENT = 10000

## 2. load the data

In [4]:
# Use only the time-series and the static dataset. 
df_desc = pd.read_csv(os.path.join(os.getcwd(),SOURCE_FOLDER, DESCRIPTION_FILE),encoding='utf-8',index_col=0)
df_data = pd.read_csv(os.path.join(os.getcwd(),SOURCE_FOLDER, DATA_FILE),encoding='utf-8', index_col=0)

# trading signal
df_signal_ma = pd.read_csv(os.path.join(os.getcwd(),TARGET_FOLDER, 'moving_average_signal.csv'), encoding='utf-8',index_col=0)
df_turtle = pd.read_csv(os.path.join(os.getcwd(),TARGET_FOLDER, 'turtle_signal.csv'), encoding='utf-8',index_col=0)

## 3. Signal trading

In [5]:
# Presentation to buy the stock and illustrate
def buy_stock(real_movement, signal, initial_money = INVESTMENT, max_buy = 1, max_sell = 1):
    """
    real_movement = Stock price time-series - pandas dataframe
    signal = Buy/Sell signals - pandas dataframe
    initial_money: Value of funds to invest (unitary monetary value, i.e. USD)
    max_buy : Quantity to buy at each positive signal
    max_sell : Quantity to sell at each negative signal
    """
    starting_money = initial_money
    states_sell = []
    states_buy = []
    current_inventory = 0  # inventory of the stocks (Check that we don't invest all)

    def buy(i, initial_money, current_inventory):
        """
        Implement the buy signal
        i: position index
        initial_money: Cash amount available at t
        current_inventory: inventory of stock - positive integer
        """
        shares = initial_money // real_movement[i]  # Get the number available
#         print "Value money", initial_money
#         print "Stock", np.isnan(real_movement[i])
#         print "Stock", real_movement.index[i]
#         print "Nb shares", shares
        if shares < 1:
            print('day nb %d -date %s: total balances %f, not enough money to buy a unit price %f'
                % (i, real_movement.index[i],  initial_money, real_movement[i]))
        else:
            if shares > max_buy:
                buy_units = max_buy
            else:
                buy_units = shares
            initial_money -= buy_units * real_movement[i]
            current_inventory += buy_units
            print('day nb %d -date %s: buy %d units at price %f, total balance %f'
                % (i, real_movement.index[i], buy_units, buy_units * real_movement[i], initial_money))
            states_buy.append(0)
        return initial_money, current_inventory

    for i in range(real_movement.shape[0] - int(0.025 * real_movement.shape[0])):
        state = signal[i]
        if state == 1:
            # Check if value is NaN
            if not np.isnan(real_movement[i]):
                initial_money, current_inventory = buy(i, initial_money, current_inventory)
                states_buy.append(i)
        elif state == -1:
            if current_inventory == 0:
                    print('day nb %d -date %s: cannot sell anything, inventory 0' % (i, real_movement.index[i]))
            else:
                if current_inventory > max_sell:
                    sell_units = max_sell
                else:
                    sell_units = current_inventory
                current_inventory -= sell_units
                total_sell = sell_units * real_movement[i]
                initial_money += total_sell
                try:
                    invest = (
                        (real_movement[i] - real_movement[states_buy[-1]])
                        / real_movement[states_buy[-1]]
                    ) * 100
                except:
                    invest = 0
                print('day nb %d -date %s, sell %d units at price %f, investment %f %%, total balance %f,'
                    % (i, real_movement.index[i], sell_units, total_sell, invest, initial_money))
            states_sell.append(i)
    invest = ((initial_money - starting_money) / starting_money) * 100
    total_gains = initial_money - starting_money
    return states_buy, states_sell, total_gains, invest

In [6]:
# Execution with turtle for one stock
states_buy, states_sell, total_gains, invest = buy_stock(df_data['PA1436583006-P'], df_turtle['PA1436583006-FTURTLE'])

day nb 10 -date 2004-01-16: cannot sell anything, inventory 0
day nb 12 -date 2004-01-20: cannot sell anything, inventory 0
day nb 16 -date 2004-01-26: cannot sell anything, inventory 0
day nb 19 -date 2004-01-29: cannot sell anything, inventory 0
day nb 20 -date 2004-01-30: cannot sell anything, inventory 0
day nb 21 -date 2004-02-02: cannot sell anything, inventory 0
day nb 33 -date 2004-02-17: cannot sell anything, inventory 0
day nb 50 -date 2004-03-11: buy 1 units at price 42.460000, total balance 9957.540000
day nb 52 -date 2004-03-15: buy 1 units at price 41.470000, total balance 9916.070000
day nb 53 -date 2004-03-16: buy 1 units at price 40.910000, total balance 9875.160000
day nb 66 -date 2004-04-02, sell 1 units at price 45.940000, investment 12.295282 %, total balance 9921.100000,
day nb 67 -date 2004-04-05, sell 1 units at price 46.140000, investment 12.784160 %, total balance 9967.240000,
day nb 68 -date 2004-04-06, sell 1 units at price 46.370000, investment 13.346370 %,

In [7]:
print(total_gains, invest)

(-324.6499999999887, -3.246499999999887)


In [8]:
# Execution with Moving Average
states_buy, states_sell, total_gains, invest = buy_stock(df_data['PA1436583006-P'], df_signal_ma['PA1436583006-FMA'])

day nb 0 -date 2004-01-02: buy 1 units at price 39.820000, total balance 9960.180000
day nb 1 -date 2004-01-05: buy 1 units at price 40.430000, total balance 9919.750000
day nb 2 -date 2004-01-06: buy 1 units at price 40.750000, total balance 9879.000000
day nb 3 -date 2004-01-07: buy 1 units at price 40.830000, total balance 9838.170000
day nb 4 -date 2004-01-08: buy 1 units at price 41.240000, total balance 9796.930000
day nb 5 -date 2004-01-09: buy 1 units at price 41.030000, total balance 9755.900000
day nb 6 -date 2004-01-12: buy 1 units at price 41.040000, total balance 9714.860000
day nb 7 -date 2004-01-13: buy 1 units at price 40.820000, total balance 9674.040000
day nb 8 -date 2004-01-14: buy 1 units at price 42.280000, total balance 9631.760000
day nb 9 -date 2004-01-15: buy 1 units at price 42.870000, total balance 9588.890000
day nb 10 -date 2004-01-16: buy 1 units at price 43.650000, total balance 9545.240000
day nb 12 -date 2004-01-20: buy 1 units at price 43.900000, tota

day nb 2097 -date 2011-12-12: total balances 2.340000, not enough money to buy a unit price 33.930000
day nb 2098 -date 2011-12-13: total balances 2.340000, not enough money to buy a unit price 33.720000
day nb 2099 -date 2011-12-14: total balances 2.340000, not enough money to buy a unit price 32.920000
day nb 2100 -date 2011-12-15: total balances 2.340000, not enough money to buy a unit price 33.070000
day nb 2101 -date 2011-12-16: total balances 2.340000, not enough money to buy a unit price 33.870000
day nb 2102 -date 2011-12-19: total balances 2.340000, not enough money to buy a unit price 32.800000
day nb 2103 -date 2011-12-20: total balances 2.340000, not enough money to buy a unit price 32.750000
day nb 2104 -date 2011-12-21: total balances 2.340000, not enough money to buy a unit price 32.780000
day nb 2105 -date 2011-12-22: total balances 2.340000, not enough money to buy a unit price 32.640000
day nb 2106 -date 2011-12-23: total balances 2.340000, not enough money to buy a u

In [10]:
print(total_gains, invest)
# PB Need to sell the stock on last day otherwise seen as losses. 

(-9997.660000000009, -99.97660000000009)


## 4. Buy-and-hold strategy.

In [30]:
NB_STOCKS = df_desc[df_desc['TYPE'] == 'Equity'].shape[0]  # Number of stocks. 
amount_per_stock = INVESTMENT/NB_STOCKS

In [44]:
total_gains, invest = 0,0
for isin in df_desc[df_desc['TYPE'] == 'Equity'].index:
    print("\n Code",isin)
    print("price t=0", df_data["-".join([isin,"P"])][0])
    print("price t=T", df_data["-".join([isin,"P"])][-1])
    print("Nb stock", amount_per_stock // df_data["-".join([isin,"P"])][0])
    return_stock = (
        df_data["-".join([isin,"P"])][-1] - df_data["-".join([isin,"P"])][0])/ df_data["-".join([isin,"P"])][0]
    print ("return", return_stock)
    total_gains += amount_per_stock * (1 + return_stock)
total_gains = total_gains - INVESTMENT
invest = total_gains/INVESTMENT
print(total_gains, invest)

('\n Code', u'PA1436583006')
('price t=0', 39.82)
('price t=T', 40.17)
('Nb stock', 62.0)
('return', 0.008789552988448052)
('\n Code', u'US22160K1051')
('price t=0', 36.32)
('price t=T', 119.02)
('Nb stock', 68.0)
('return', 2.2769823788546253)
('\n Code', u'US4581401001')
('price t=0', 32.16)
('price t=T', 25.955)
('Nb stock', 77.0)
('return', -0.19294154228855717)
('\n Code', u'AN8068571086')
('price t=0', 27.33)
('price t=T', 90.11)
('Nb stock', 91.0)
('return', 2.2971094035858033)
(10974.849482850797, 1.0974849482850797)
